# Echo-B 1B4 (Basemodel Training)
This model is a custom 1.4B model containing
- 96 layers
- 1024 embedding size

It was initialized using the original RWKV trainer here : https://github.com/PicoCreator/RWKV-LM-LoRA/blob/picocreator-init-memory-experiment/notebook/echo-B-1B4-training.ipynb

The goal of this model training is to exceed RWKV model memory, with its increased layer size.

While the original enwiki-v1 replication of 1.5B did not fully match the finetuned version memory capacity, it managed to match raven 1.5B performance levels.
This helps prove that the training method used worked, and does not require training a full "pile+" model from scratch.

As such adjustments were made in the training process (based on learnings in the original), and applied to this iteration. Where we will be training a 96 layer varient from scratch.

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps

# Basic Setup

In [ ]:
# # First lets get the blank init model, these init model was generated
# # using the original RWKV-LM repo (as at this point of writing, this repo cannot init a model)
# #
# # As such I have preinitialized these blank models and uploaded them to HF for convinence
# #
# # Comment back in this cell if you need it
# !mkdir -p ../../../model/
# !mkdir -p ../../../datapath/
# !mkdir -p ../../../checkpoint/
# !rm -rf ../../../model/Echo-A-1B4-Init.pth
# !cd ../../../model/ && wget https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-B-1B4-Init.pth
# !ls -alh ../../../model/Echo-A-1B4-Init.pth

In [ ]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="Echo-B-1B4"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4neo/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

## Stage 1 : Foundation model training

In [ ]:
# Lets preload the requried dataset (enwiki_100k)
!cd "{TRAINER_DIR}" && \
    python3 preload_dataset.py "{NOTEBOOK_DIR}/Echo-A-1B4-enwiki.yaml"

In [ ]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    python new_train.py fit \
        -c "{NOTEBOOK_DIR}/Echo-A-1B4-enwiki.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Foundation (train-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" 

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    && python export_checkpoint.py "../checkpoint/Echo-A-1B4-enwiki/last.ckpt"

In [ ]:
# Lets move and save this model
!cd "{TRAINER_DIR}" && cp ./checkpoint/Echo-A-1B4-enwiki/last.ckpt/rwkv_model.pth ./model/Echo-A-1B4-Stage1.pth
!cd "{TRAINER_DIR}" && ls -alh ./model/Echo-A-1B4-Stage1.pth

In [ ]:
# Lets do a quick dragon prompt validation
!cd "{TRAINER_DIR}" && python3 dragon_test.py ../model/Echo-A-1B4-Stage1.pth "cuda fp32"

# Stage 2 : Instruct Tuning

In [ ]:
# Lets preload the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_dataset.py "{NOTEBOOK_DIR}/Echo-A-1B4-instruct.yaml"

In [ ]:
# Start the instruct finetuning
!cd "{TRAINER_DIR}" && \
    python new_train.py fit \
        -c "{NOTEBOOK_DIR}/Echo-A-1B4-instruct.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Instruct (train-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    && python export_checkpoint.py "../checkpoint/Echo-A-1B4-instruct/last.ckpt"

In [ ]:
# Lets move and save this model
!cd "{TRAINER_DIR}" && cp ./checkpoint/Echo-A-1B4-instruct/last.ckpt/rwkv_model.pth ./model/Echo-A-1B4-Stage2.pth
!cd "{TRAINER_DIR}" && ls -alh ./model/Echo-A-1B4-Stage2.pth

In [ ]:
# Lets do a quick dragon prompt validation
!cd "{TRAINER_DIR}" && python3 dragon_test.py ../model/Echo-A-1B4-Stage2.pth "cuda fp32"

In [ ]:
# Lets do a quick memory test
# (We dun expect this to work, as we have not finetune for memory recall, but its a baseline)
!python3 ./memory_script/eval_memory_guided.py "{PROJECT_DIR}/model/Echo-A-1B4-Stage2.pth"